In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import json

# Load and prepare the dataset
file_path = 'goal_ticks.csv'

# Define column names based on the provided structure
column_names = [
    "kart_ID", "ball_X", "ball_Z", "ball_aim_X", "ball_aim_Z", "previous_X", "previous_Z",
    "dist_to_ball", "kart_X", "kart_Z", "vel_X", "vel_Z", "speed", "steer", "accel", "brake", "skid", "time", "goal"
]

df = pd.read_csv(file_path, header=None, names=column_names)

# Extract features and labels
X = df.iloc[:, :-1].values  # All columns except the last one (goal)
y = df['goal'].values  # The 'goal' column as binary

# Normalize all features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler parameters for later use
scaler_params = {
    "mean": scaler.mean_.tolist(),
    "scale": scaler.scale_.tolist()
}

print("scaler params: ")
print(scaler_params)


with open('scaler_parameters.json', 'w') as f:
    json.dump(scaler_params, f)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Keeping the target as a 2D tensor for BCELoss

# Define the neural network architecture
class ScoringModel(nn.Module):
    def __init__(self, num_inputs=18):  # Set to 18 inputs
        super(ScoringModel, self).__init__()
        self.layer1 = nn.Linear(num_inputs, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, 64)
        self.layer5 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)  # Binary classification
        self.leaky_relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.leaky_relu(self.layer1(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer2(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer3(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer4(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer5(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.output_layer(x))
        return x

# Initialize the model with 18 inputs
model = ScoringModel(num_inputs=18)

# Set up the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Prepare data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training function
def train_model(train_loader, model, criterion, optimizer, num_epochs=300):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        if (epoch + 1) % 10 == 0:  # Print every 10 epochs
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Validation function
def validate_model(val_loader, model, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    print(f'Validation Loss: {total_loss / len(val_loader):.4f}')

# Train and validate the model
train_model(train_loader, model, criterion, optimizer, 300)
validate_model(val_loader, model, criterion)

# Save the trained model for deployment
model.eval()

example_input = torch.rand(1, X_train.shape[1])  # Generate a random example input
traced_script_module = torch.jit.trace(model, example_input)
traced_script_module.save("soccer_ai_model.pt")

# Print the weights of the output layer after training
print("Weights of the output layer after training:")
print(model.output_layer.weight)
print("Bias of the output layer after training:")
print(model.output_layer.bias)


scaler params: 
{'mean': [0.5, 3.5483308625738403, 3.9604762566431786, 3.6175693510669316, 5.422774172115293, -1.1462591634198658, -4.480996968547831, 25.186589012681146, -1.144882902324445, -4.296661955811389, -0.014670579882463137, 1.9652008043429574, 4.962660554657229, 0.010147477157524712, 0.5195828741190468, 0.2450117460414315, 0.0, 20086306.228910517], 'scale': [0.5, 22.031614660294085, 33.96606929924976, 22.603484641646038, 34.16584984976145, 19.330746270345855, 30.65604063091596, 21.788951195422914, 19.32594819227115, 30.734864349813883, 4.697431817628639, 6.127791205606288, 5.642693029843885, 0.698035013591742, 0.3061657919208652, 0.43009416450721644, 1.0, 7837048.656515318]}
Epoch 10/300, Loss: 0.0876
Epoch 20/300, Loss: 0.0746
Epoch 30/300, Loss: 0.0175
Epoch 40/300, Loss: 0.0167
Epoch 50/300, Loss: 0.0200
Epoch 60/300, Loss: 0.0998
Epoch 70/300, Loss: 0.0201
Epoch 80/300, Loss: 0.0160
Epoch 90/300, Loss: 0.0733
Epoch 100/300, Loss: 0.0201
Epoch 110/300, Loss: 0.0760
Epoch 1